In [ ]:

#  Demonstrating using Cypher to create bi-directional edges ..
#



#  Setup stuff: Connectivity

In [ ]:
#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph_BD"

print("--")


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [47]:

#  DELETE ONE SPECIFIC GRAPH ONLY

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name = l_database.name).graphs_in_database():
      if ((l_database.name == DB_NAME) and (l_graph.name == GRAPH_NAME)):   
         l_handle = my_client.get_database(name = l_database.name).get_graph_by_id(id = l_graph.graph_id)
         l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "   GRAPH Version: ", l_graph.version, "   Graph Name: ", l_graph.name)

print("--")


GRAPH ID:  5NUwpV8GmVPB2FWCGYwWc32GQrPySTb38Cw7A29kQTvx    GRAPH Version:  1    Graph Name:  my_graph_BC
GRAPH ID:  6GYEDQxA2ewmmGV53Vb7GDBx5U9osUzBctJhh6auAdG3    GRAPH Version:  92    Graph Name:  my_graph_BB
GRAPH ID:  EcCpQwrXXjTA5QjWk8dgLC5Sq1AEFwRQksAWjTYHfZWK    GRAPH Version:  8    Graph Name:  my_graph
--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [48]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph_BD, JDVJf15v1gHKbHxJsnMAF11V4kQAvbLHeWuGBn4m5ngb, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
import pandas as pd

print("--")

In [31]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
   ["LAX", "Los Angeles"             , "Airport"],
   ["SLC", "Salt Lake City"          , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


,airport_code,airport_name,LABEL
0,MKE,Milwaukee,Airport
1,ORD,Chicago O-Hare,Airport
2,SJC,San Jose,Airport
3,DEN,Denver,Airport
4,LAX,Los Angeles,Airport
5,SLC,Salt Lake City,Airport


In [32]:

#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],
   ["LAX", "ORD", 2029, 1, "FLIES_TO" ],
   ["SLC", "ORD", 1394, 1, "FLIES_TO" ],
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(20)


,START_ID,END_ID,DISTANCE,NUM_HOPS,TYPE
0,MKE,ORD,66,1,FLIES_TO
1,DEN,ORD,886,1,FLIES_TO
2,SJC,ORD,1829,1,FLIES_TO
3,LAX,ORD,2029,1,FLIES_TO
4,SLC,ORD,1394,1,FLIES_TO


In [49]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

--


#  Diagnostics:

In [42]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


          0/? [?op/s]

6

          0/? [?op/s]

5

In [44]:

#  What's in the vertices/nodes ..

l_query  = """
   MATCH ( n ) 
   RETURN n
   """.format()

l_result = my_graph.query_unpaginated(l_query)

print(tabulate(l_result, headers="keys", tablefmt="psql", showindex = False))


          0/? [?op/s]

+-----------------+-------------+-----------+------------------+------------------+--------------+
|           id(n) | labels(n)   | n.LABEL   | n.airport_code   | n.airport_name   | n datatype   |
|-----------------+-------------+-----------+------------------+------------------+--------------|
| 281474976710656 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node         |
| 281474976710657 | ['Airport'] | Airport   | SJC              | San Jose         | node         |
| 562949953421314 | ['Airport'] | Airport   | LAX              | Los Angeles      | node         |
| 562949953421315 | ['Airport'] | Airport   | SLC              | Salt Lake City   | node         |
| 562949953421313 | ['Airport'] | Airport   | DEN              | Denver           | node         |
| 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        | node         |
+-----------------+-------------+-----------+------------------+------------------+--------------+


In [52]:

#  What's in the edge .. 

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN r
   // RETURN  n.id AS FROM, m.id AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   """.format()

l_result = my_graph.query_unpaginated(l_query)

print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))


          0/? [?op/s]

+------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
|   id(endNode(r)) | labels(endNode(r))   |           id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | r datatype   |
|------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------|
|  281474976710656 | ['Airport']          |               0 | FLIES_TO  |           66 |            1 | FLIES_TO |                  0 | ['Airport']            | edge         |
|  281474976710656 | ['Airport']          |               1 | FLIES_TO  |         1829 |            1 | FLIES_TO |                  1 | ['Airport']            | edge         |
|  281474976710657 | ['Airport']          | 281474976710658 | FLIES_TO2 |          nan |          nan | nan      |    28

In [ ]:

#  Sanity check look at the graph ..
#

l_query  = """

   MATCH (n) - [ r ] -> (m)
   RETURN n, r, m
   
   """.format()

l_result = my_graph.query(l_query, contextualize = True)

l_result.view()


#  Use Cypher to make reverse edges:

In [36]:

#  Cypher command to make reverse edges ..
#

l_query  = """

   MATCH (n: Airport) - [r: FLIES_TO] -> (m: Airport) 
   WITH n, m 
   CREATE (m) - [e: FLIES_TO] -> (n)
   
   """.format()

l_result = my_graph.query_unpaginated(l_query)

print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))


#  Results pre and post,
#
#     +------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
#     |   id(endNode(r)) | labels(endNode(r))   |           id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | r datatype   |
#     |------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------|
#     |  281474976710656 | ['Airport']          | 281474976710656 | FLIES_TO  |         1829 |            1 | FLIES_TO |    281474976710657 | ['Airport']            | edge         |
#     |  281474976710656 | ['Airport']          | 562949953421315 | FLIES_TO  |         1394 |            1 | FLIES_TO |    562949953421315 | ['Airport']            | edge         |
#     |  281474976710656 | ['Airport']          | 562949953421314 | FLIES_TO  |         2029 |            1 | FLIES_TO |    562949953421314 | ['Airport']            | edge         |
#     |  281474976710656 | ['Airport']          | 562949953421312 | FLIES_TO  |           66 |            1 | FLIES_TO |    562949953421312 | ['Airport']            | edge         |
#     |  281474976710656 | ['Airport']          | 562949953421313 | FLIES_TO  |          886 |            1 | FLIES_TO |    562949953421313 | ['Airport']            | edge         |
#     +------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
#     
#     +------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
#     |   id(endNode(r)) | labels(endNode(r))   |           id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | r datatype   |
#     |------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------|
#     |  562949953421312 | ['Airport']          |               0 | FLIES_TO  |           66 |            1 | FLIES_TO |                  0 | ['Airport']            | edge         |
#     |  562949953421312 | ['Airport']          |               1 | FLIES_TO  |         2029 |            1 | FLIES_TO |                  1 | ['Airport']            | edge         |
#     |  562949953421312 | ['Airport']          | 281474976710656 | FLIES_TO  |         1394 |            1 | FLIES_TO |    281474976710656 | ['Airport']            | edge         |
#     |  562949953421312 | ['Airport']          | 562949953421312 | FLIES_TO  |         1829 |            1 | FLIES_TO |    562949953421313 | ['Airport']            | edge         |
#     |  562949953421312 | ['Airport']          | 562949953421313 | FLIES_TO  |          886 |            1 | FLIES_TO |    562949953421314 | ['Airport']            | edge         |
#     |  562949953421313 | ['Airport']          | 562949953421314 | FLIES_TO  |          nan |          nan | nan      |    562949953421312 | ['Airport']            | edge         |
#     |  562949953421314 | ['Airport']          | 562949953421315 | FLIES_TO  |          nan |          nan | nan      |    562949953421312 | ['Airport']            | edge         |
#     |  281474976710656 | ['Airport']          | 562949953421316 | FLIES_TO  |          nan |          nan | nan      |    562949953421312 | ['Airport']            | edge         |
#     |                1 | ['Airport']          | 562949953421317 | FLIES_TO  |          nan |          nan | nan      |    562949953421312 | ['Airport']            | edge         |
#     |                0 | ['Airport']          | 562949953421318 | FLIES_TO  |          nan |          nan | nan      |    562949953421312 | ['Airport']            | edge         |
#     +------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+



          0/? [?op/s]

In [51]:


xxx = "FLIES_TO2"

l_query  = """

   MATCH (n: Airport) - [r: FLIES_TO] -> (m: Airport) 
   WITH n, m 
   CREATE (m) - [e: {0}] -> (n)
   
   """.format(xxx)

l_result = my_graph.query_unpaginated(l_query)

print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))



          0/? [?op/s]